In [ ]:
from __future__ import annotations

from pathlib import Path

import numpy as np

import xclim
from xclim import sdba
from xclim.testing import open_dataset

ds = open_dataset(
    "sdba/CanESM2_1950-2100.nc", cache=True, cache_dir=Path("~/data/xclim/")
)
tx = ds.tasmax.sel(time=slice("1950", "1980")).isel(location=0)
q = np.linspace(0, 1, 50)
arr = tx.values
arr_m1 = arr.copy()
arr_m2 = arr.copy()

# do some sort of computation to make sure it's compiled.
np.square(
    sdba.nbutils._sortquantile(np.copy(arr_m1), q)
    - sdba.nbutils.nan_quantile(np.copy(arr_m2), q)
).sum()

In [ ]:
sdba.nbutils.nan_quantile.dump()

In [ ]:
%%timeit
# sortquantile implementation
sdba.nbutils._sortquantile(arr_m1, q)

In [ ]:
%%timeit arr_m3 = np.copy(arr_m2)
# 1d numba-compiled version of xclim.core.utils.nan_quantile (cheating)
sdba.nbutils.nan_quantile(arr_m3, q)

In [ ]:
%%timeit
# 1d numba-compiled version of xclim.core.utils.nan_quantile
sdba.nbutils.nan_quantile(np.copy(arr_m2), q)

In [ ]:
%%timeit
# Abel's implementation
xclim.core.utils.nan_quantile(np.copy(arr_m2), q)

In [ ]:
%%timeit arr_m3 = np.copy(arr_m2)
# Abel's implementation (in-place (cheating)).
xclim.core.utils.nan_quantile(arr_m3, q)

In [ ]:
arr_m3 = np.copy(arr_m2)

In [ ]:
%%timeit
# Abel's implementation (in-place, sorted beforehand (super cheating!)).
xclim.core.utils.nan_quantile(arr_m3, q)

In [ ]:
import dask
import xarray as xr

ds2 = xr.open_dataset(
    "~/data/xclim/tasmax_day_CanESM5_ssp245_r1i1p1f1_gn_20150101-21001231.nc"
)

ds_sel2 = ds2.isel(lat=0, lon=0).tasmax.compute()

In [ ]:
arr2 = ds_sel2.values.copy()
percent_of_nans = 0.0
num_of_nans = int(percent_of_nans * arr2.size)
random_indices = np.random.randint(0, arr2.shape[0], size=num_of_nans)
arr2[random_indices] = np.nan

In [ ]:
%%timeit
xclim.core.utils.nan_quantile(arr2, q)

In [ ]:
%%timeit
sdba.nbutils._sortquantile(arr2, q)

In [ ]:
arr2.shape

In [ ]:
q